
*   Computing Platforms: Set up the Workspace for Machine Learning Projects.  https://ms.pubpub.org/pub/computing
*  Machine Learning for Predictions. https://ms.pubpub.org/pub/ml-prediction
* Machine Learning Packages: https://scikit-learn.org/stable/


# Part I: Import and Inspect Data

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Rising-Stars-by-Sunshine/YutongQuan_STATS201_FinalProject/main/data/Queried_Data/Aave_Queried.csv')
df.head()

,timestamp,open,high,low,close,volume,market cap
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109


In [3]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'market cap'], dtype='object')

In [4]:
df.dtypes

timestamp      object
open          float64
high          float64
low           float64
close         float64
volume        float64
market cap    float64
dtype: object

# Part II: Prepare the Y varible for Regression

## 2.1. Write functions to calculte the Y variable for Regression 

*(skip the step if the Y variable already exists)*

In [5]:
df['daily return'] = df['close'].pct_change()
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345


## 2.2. Make Sure that the Data Type of Y is "numeric"

In [6]:
df['daily return']=pd.to_numeric(df['daily return'])
df.dtypes

timestamp        object
open            float64
high            float64
low             float64
close           float64
volume          float64
market cap      float64
daily return    float64
dtype: object

# Part III: Prepare the Y variable for Classification

reference:

https://datatofish.com/if-condition-in-pandas-dataframe/ *italicized text*

In [7]:
#@title Define the Positive Threshold
cut = 0 #@param {type:"number"}


## 3.1. Method 1: If function

In [8]:
df['positive'] = df['daily return'] >= cut
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,False
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,False
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,True
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,False
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,True


In [9]:
df.loc[(df['daily return'] >= cut), 'positive'] = 1
df.loc[(df['daily return'] <cut), 'positive'] = 0
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,False
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,0
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,1
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,0
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,1


## 3.2. Method 2: Lambda function

notes: the best method that I suggest

In [10]:
df['positive'] = df['daily return'].apply(lambda x: 1 if x>= cut else 0)
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,0
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,0
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,1
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,0
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,1


# Part IV: Create the X variables

## 4.1. Shift the Y to get past values

reference:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [11]:
# generate a new variable as the previous 1 observable of your Y variable for regression
df['daily return_past'] =df['daily return'].shift(1)
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive,daily return_past
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,0,NaN
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,0,NaN
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,1,-0.003241
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,0,0.021138
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,1,-0.022293


## 4.2. Calculate the Moving Averages

references: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

https://towardsdatascience.com/moving-averages-in-python-16170e20f6c

In [12]:
#@title Define the Window
window = 30 #@param {type:"number"}


In [13]:
df['daily return_past_ma30']=df['daily return_past'].rolling(window=window,min_periods=1).mean()
df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive,daily return_past,daily return_past_ma30
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,0,NaN,NaN
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,0,NaN,NaN
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,1,-0.003241,-0.003241
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,0,0.021138,0.008948
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,1,-0.022293,-0.001465


# Part V Train and Test Split

*reference*:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [14]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit()
print(tss)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [15]:
# change the train and test split parameters 
tss = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)

In [16]:
for train_idx, test_idx in tss.split(df):
    print("TRAIN:", train_idx, "TEST:", test_idx)

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [17]:
train_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [18]:
test_idx

array([525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550,
       551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563,
       564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576,
       577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589,
       590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602,
       603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615,
       616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628,
       629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641,
       642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654,
       655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667,
       668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680,
       681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693,
       694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 70

In [19]:
train_df = df.filter(items=train_idx, axis=0)
test_df =  df.filter(items=test_idx, axis=0)

In [20]:
train_df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive,daily return_past,daily return_past_ma30
0,2022/12/10,61.5,61.9,61.3,61.7,11997.611,11997.611,NaN,0,NaN,NaN
1,2022/12/9,62.9,63.4,61.0,61.5,92191.065,92191.065,-0.003241,0,NaN,NaN
2,2022/12/8,61.4,63.5,60.4,62.8,78099.435,78099.435,0.021138,1,-0.003241,-0.003241
3,2022/12/7,64.0,64.1,60.1,61.4,138823.250,138823.250,-0.022293,0,0.021138,0.008948
4,2022/12/6,64.4,65.6,62.7,64.0,77089.109,77089.109,0.042345,1,-0.022293,-0.001465


In [21]:
test_df.head()

,timestamp,open,high,low,close,volume,market cap,daily return,positive,daily return_past,daily return_past_ma30
525,2021/7/3,234.05,267.08,227.75,259.42,275045.4219,275045.4219,-0.063466,0,-0.101291,-0.002212
526,2021/7/2,230.18,234.96,215.62,234.12,183917.2291,183917.2291,-0.097525,0,-0.063466,-0.005586
527,2021/7/1,250.96,253.02,227.22,230.17,234518.7354,234518.7354,-0.016872,0,-0.097525,-0.008419
528,2021/6/30,238.03,252.58,219.71,251.08,273789.0701,273789.0701,0.090846,1,-0.016872,-0.010532
529,2021/6/29,227.32,253.32,226.04,238.22,280295.6893,280295.6893,-0.051219,0,0.090846,-0.006436


# Part VI Prepare the Train and Test Data for Classification and Regression

## 6.1. Classification

### 6.1.1 Define the columns (Y, X) for Classification 

In [22]:
cols_C = ['positive','daily return_past_ma30']

### 6.1.2 Define the Data Frame of Train and Test Data for Classification

In [23]:
df_C_train = train_df[cols_C]
df_C_test = test_df[cols_C]

### 6.1.3 Export the Train and Test Data for Classification

In [24]:
df_C_train.head()

,positive,daily return_past_ma30
0,0,NaN
1,0,NaN
2,1,-0.003241
3,0,0.008948
4,1,-0.001465


In [25]:
df_C_train.to_csv('Classification_Train.csv')

In [26]:
df_C_test.head()

,positive,daily return_past_ma30
525,0,-0.002212
526,0,-0.005586
527,0,-0.008419
528,1,-0.010532
529,0,-0.006436


In [27]:
df_C_test.to_csv('Classification_Test.csv')

## 6.2 Regression

### 6.2.1. Define the columns (Y, X) for Regression

In [28]:
cols_R = ['daily return','daily return_past_ma30']

### 6.2.2. Define the Data Frame of Train and Test Data for Regression

In [29]:
df_R_train = train_df[cols_R]
df_R_test = test_df[cols_R]

### 6.2.3. Export the Train and Test Data for Regression

In [30]:
df_R_train.head()

,daily return,daily return_past_ma30
0,NaN,NaN
1,-0.003241,NaN
2,0.021138,-0.003241
3,-0.022293,0.008948
4,0.042345,-0.001465


In [31]:
df_R_train.to_csv('Regression_Train.csv')

In [32]:
df_R_test.head()

,daily return,daily return_past_ma30
525,-0.063466,-0.002212
526,-0.097525,-0.005586
527,-0.016872,-0.008419
528,0.090846,-0.010532
529,-0.051219,-0.006436


In [33]:
df_R_test.to_csv('Regression_Test.csv')

In [39]:
df.to_csv('Aave_Processed.csv')